In [15]:
import torch

# Starter tensors
a = torch.arange(24).reshape(2, 3, 4).float()
b = torch.randn(2, 3, 4)

# Tasks:
# 1. Compute mean and standard deviation of `a` along last dim.
# 2. Normalize `a` along dim=1 (so sum over dim=1 equals 1).
# 3. Compute matrix multiplication between a[0] and b[0]^T (transpose b[0]).
# 4. Flatten a into shape (3, 8) *without* using view(-1, 8).
# 5. Permute a to shape (4, 2, 3) — double-check with a.shape.
mean_a = torch.mean(a, dim=-1)
std_a = torch.var(a, dim= -1)

second = torch.softmax(a, dim=1)

third = torch.matmul(a[0], torch.transpose(b[0], -1, -2)) # output should be of size 3 3 
# shape of a is 2, 3, 4 
fourth = a.reshape(3,8)

a = a.permute(2,0,1)

# b[0] shape is 3, 4 transpose -1,and -2 

In [58]:
x = torch.randn(5, 5)

# Tasks:
# 1. Replace all negative values with their absolute values.
# 2. Create a mask of all values > mean(x), and set them to 0.
# 3. Extract the diagonal elements using advanced indexing.
# 4. Normalize each row so that its L2 norm = 1.
# 5. Write a single-line expression to reverse the rows of x.
# 1 
mask = x<0
y = torch.where(x<0, -x,x)

x_mean = torch.mean(x)
y = torch.where(x>x_mean, 0, x)

# extract the diagonal 
y = torch.arange(0,5)

diag = x[y,y]
print(diag)
# normalise each row that L2 norm is 1 
#x2 sum is 1 
a= torch.arange(x.shape[0] )
mask = torch.zeros_like(x, dtype=torch.bool)
mask[a, a] = True
x[mask]

tensor([-0.4919,  1.3355, -0.4124, -0.1862,  0.0107])


tensor([-0.4919,  1.3355, -0.4124, -0.1862,  0.0107])

In [76]:


row_norm = torch.sqrt(torch.sum(x*x, dim=1, keepdim=True))
x = x/row_norm 
x

tensor([[-0.1931, -0.1491, -0.6344, -0.6825,  0.2688],
        [-0.2968,  0.6635, -0.4444, -0.4904, -0.1837],
        [-0.7733,  0.3450, -0.1551,  0.2395,  0.4488],
        [ 0.7648,  0.5879, -0.0982, -0.1344,  0.2045],
        [-0.1166,  0.2184, -0.7496, -0.6138,  0.0062]])

In [78]:
x.flip(1)

tensor([[ 0.2688, -0.6825, -0.6344, -0.1491, -0.1931],
        [-0.1837, -0.4904, -0.4444,  0.6635, -0.2968],
        [ 0.4488,  0.2395, -0.1551,  0.3450, -0.7733],
        [ 0.2045, -0.1344, -0.0982,  0.5879,  0.7648],
        [ 0.0062, -0.6138, -0.7496,  0.2184, -0.1166]])

In [132]:
x = torch.randn(3, 3, requires_grad=True)

# Tasks:
# 1. Compute y = (x ** 2).sum() and backprop to get dy/dx.
# 2. Compute z = torch.exp(x) + torch.sin(x); find dz/dx.
# 3. Create tensors a(3,4,5) and b(4,5) and add them via broadcasting.
# 4. Manually compute softmax(x, dim=1) using tensor ops only (no torch.softmax).
# 5. Implement cross entropy loss manually given logits and one-hot targets.
y = torch.sum(x**2)
y_grad = torch.autograd.grad(y, x)[0]
print(y_grad.shape)

z = torch.exp(x) + torch.sin(x)
z_grad, = torch.autograd.grad(z, x, grad_outputs=torch.ones_like(z))


# 3 
a = torch.randn(3,4, 5)
b = torch.randn(4,5)
c = a+b
# this work only if the last two are same, but no broadcasting if first two are smae
#So yeah — broadcasting only cares that the last dimensions align, not the first ones.
c.shape

# 4 
x = torch.randn(3, 3, requires_grad=True)
# make this nuemrically stable 
x_max = torch.max(x, dim= 1, keepdim= True).values
x = x-x_max
x_exp = torch.exp(x) # 3, 3
x_sum = torch.sum(x_exp, dim =1, keepdim= True)
soft_x = x_exp/ x_sum 
soft_x.shape

# 5. Implement cross entropy loss manually given logits and one-hot targets.
logits = torch.rand(3, 5)
# 3 samples, 5 classes 

one_hot = torch.zeros_like(logits)
a = torch.arange(3)
a = a.unsqueeze(1)
print(a.shape)
print(one_hot.shape)
one_hot.scatter_(1, a, 1 )
print(one_hot)
# CE loss 
CE_loss = - torch.mean(torch.log(torch.softmax(logits,dim= 1 ))*one_hot, dim = 1 )
print(CE_loss)

torch.Size([3, 3])
torch.Size([3, 1])
torch.Size([3, 5])
tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])
tensor([0.2583, 0.3368, 0.2760])


In [184]:
# 🧬 Level 4 – Spatial Thinking (Matrix Ops & Geometry)
coords = torch.randn(10, 3)
rot = torch.tensor([
    [0.0, -1.0, 0.0],
    [1.0,  0.0, 0.0],
    [0.0,  0.0, 1.0]
])

# Tasks:
# 1. Rotate all coords using batched matrix multiplication.
# 2. Compute pairwise Euclidean distance between all points.
# 3. Compute pairwise cosine similarity using broadcasting.
# 4. Implement k-nearest neighbors (no loops).
# 5. Center all points around origin, then rescale them to unit variance.


y = torch.bmm(coords.unsqueeze(0), rot.unsqueeze(0)).squeeze(0)
# 2. Compute pairwise Euclidean distance between all points.
#y = coords[:, None, :] - coords[None, : , :]
# coords: (N, D)
x_norm = (coords ** 2).sum(dim=1, keepdim=True)   # (N, 1)
# pairwise squared distances
dists_sq = x_norm + x_norm.T - 2 * coords @ coords.T
# clamp and sqrt for clean distances
dists = torch.sqrt(torch.clamp(dists_sq, min=0.0))

# 3 cosine similarity 
coords = coords / coords.norm(dim=1, keepdim=True)
cos_sim = torch.matmul(coords, coords.transpose(-1,-2))
print(cos_sim.shape)


# k nearest neighbors for coords 
# for each coord top k nearest neighbors 
k = 3 
x_norm = (coords ** 2).sum(dim=1, keepdim=True) 
dist = x_norm + torch.transpose(x_norm, -1, -2) - 2 * torch.matmul(coords, torch.transpose(coords, -1, -2))
dist.shape
# choose top k from these distances for each row 
# either fill the diagonal with inf or chooose k+1 because the coord itself is the first closest point in it 
dist.fill_diagonal_(float('inf'))                    # ignore self

top_k_values, top_k_indices = torch.topk(dist, k, dim=1, largest= False) # for sorting 
# for each row choose the top k in column? 


## col5 
x_mean = torch.mean(x, dim =0 , keepdim=True)
x_new = x-x_mean
x_std = torch.std(x, dim=0, keepdim=True)
x_new= x_new/ x_std

torch.Size([10, 10])


In [219]:
#🤯 Level 5 – “Why did I sign up for this?” (Pure Tensor Wizardry)
# Build a mini 2-layer neural net *from scratch*:
# Input: (batch_size, 5), Hidden: 4, Output: 2
# Activation: ReLU
# Loss: MSE between predicted and target

batch_size = 8
x = torch.randn(batch_size, 5)
y = torch.randn(batch_size, 2)

W1 = torch.randn(5, 4, requires_grad=True)
b1 = torch.zeros(4, requires_grad=True)
W2 = torch.randn(4, 2, requires_grad=True)
b2 = torch.zeros(2, requires_grad=True)

# Tasks:
# 1. Forward pass using only tensor ops (no nn.Module).
# 2. Compute loss = ((y_pred - y) ** 2).mean()
# 3. Backprop manually using autograd.
# 4. Update weights using gradient descent with lr=0.01.

f = torch.relu(x@W1+b1)
output = f@W2 +b2 
#f.shape # should be 8 x 2 

loss = torch.mean((output-y)**2, dim=(0,1))


W1_grad,  = torch.autograd.grad(loss,W1, retain_graph=True)

W2_grad,  = torch.autograd.grad(loss,W2,  retain_graph=True)
b2_grad, = torch.autograd.grad( loss,b2,  retain_graph=True)
b1_grad, = torch.autograd.grad( loss,b1,  retain_graph=True)


lr = torch.tensor(0.01, dtype=torch.float16)
with torch.no_grad():
    b1 -= lr*b1_grad
    b2 -= lr*b2_grad
    W1 -= lr*W1_grad
    W2 -= lr*W2_grad
    



In [231]:
# batchnorm 
# X (batch_size, features)
x = torch.randn(8, 5)

x_mean = torch.mean(x, dim = 0, keepdim= True)
x_var = torch.var(x, dim=0, keepdim=True)
x = (x-x_mean)/ torch.sqrt(x_var+1e-15)
gamma = torch.ones(1, 5)
beta = torch.zeros(1, 5)

y = gamma * x_norm + beta

In [249]:
class MyGradient(torch.autograd.Function):
    # this does not need a self arggument
    @staticmethod 
    def forward (ctx, input):
        ctx.save_for_backward(input)
        # fx = x 3
        return input**3 
    @staticmethod 
    def backward(ctx, grad_output):
        x, = ctx.saved_tensors()
        return grad_output*3*x*x+0.5 
x = torch.randn(4, requires_grad=True)
y = torch.sum(x**3)
y.backward()
print(x.grad)
f = MyGradient()
g = f.apply(x)
print(x)


tensor([3.8215e+00, 2.4494e-04, 6.4822e+00, 8.9441e-01])
tensor([ 1.1286, -0.0090,  1.4699,  0.5460], requires_grad=True)


/var/folders/9t/78vgfzf94l3f7wnvfk1ng3lc0000gq/T/ipykernel_66975/1635869463.py:16: DeprecationWarning: <class '__main__.MyGradient'> should not be instantiated. Methods on autograd functions are all static, so you should invoke them on the class itself. Instantiating an autograd function will raise an error in a future version of PyTorch.
  f = MyGradient()


In [252]:
# ⚙️ Level 7 — “Advanced Pain but Beautiful”
# 1. Custom Autograd Function: SoftClipped ReLU

# x,0.01⋅tanh(x),x>0 x≤0	​

class CustomAutograd(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        y = torch.where(input>0, input, 0.01*torch.tanh(input))
        return y 
    @staticmethod
    def backward(ctx, grad_output):
        (x,) = ctx.saved_tensors
        # derivative piecewise: 1 for x>0, 0.01*(1 - tanh(x)^2) otherwise
        neg_grad = 0.01 * (1 - torch.tanh(x).pow(2))
        df_dx = torch.where(x > 0, torch.ones_like(x), neg_grad)
        grad_input = grad_output * df_dx
        return grad_input


In [283]:
# linear regression training loop 
x = torch.randn(5, 10)
# bacthsize, features 
y = (3 * x + 2 + torch.randn_like(x)*0.1).mean(dim=1, keepdim=True)  # (5,1)

print(y.shape)
model = torch.nn.Linear(10,1)
loss = torch.nn.MSELoss()
optim = torch.optim.SGD(model.parameters(),lr = 0.01)
for _ in range(10):
    y_hat = model(x)
    print(y_hat.shape) # 5,1 
    
    loss_ = loss(y,y_hat )
    print(loss_)
    optim.zero_grad()
    loss_.backward()
    optim.step()
    


    

torch.Size([5, 1])
torch.Size([5, 1])
tensor(4.5710, grad_fn=<MseLossBackward0>)
torch.Size([5, 1])
tensor(4.0679, grad_fn=<MseLossBackward0>)
torch.Size([5, 1])
tensor(3.6332, grad_fn=<MseLossBackward0>)
torch.Size([5, 1])
tensor(3.2568, grad_fn=<MseLossBackward0>)
torch.Size([5, 1])
tensor(2.9304, grad_fn=<MseLossBackward0>)
torch.Size([5, 1])
tensor(2.6466, grad_fn=<MseLossBackward0>)
torch.Size([5, 1])
tensor(2.3993, grad_fn=<MseLossBackward0>)
torch.Size([5, 1])
tensor(2.1833, grad_fn=<MseLossBackward0>)
torch.Size([5, 1])
tensor(1.9941, grad_fn=<MseLossBackward0>)
torch.Size([5, 1])
tensor(1.8279, grad_fn=<MseLossBackward0>)
